In [4]:
import gradio as gr
from langchain_groq import ChatGroq
from langchain_experimental.plan_and_execute import PlanAndExecute, load_agent_executor, load_chat_planner
from langchain.agents import Tool
from pydantic import BaseModel, Field
from typing import List
import os
import json

os.environ["GROQ_API_KEY"] = "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"

class AgentResponse(BaseModel):
    answer: str = Field(..., description="The final answer to the user's query")
    steps: List[str] = Field(default_factory=list, description="Steps taken by the agent")

llm = ChatGroq(
    api_key=os.environ["GROQ_API_KEY"],
    model_name="llama3-70b-8192",
    temperature=0.7
)

def calculator(expression: str) -> str:
    try:
        result = eval(expression, {"__builtins__": {}}, {})
        return json.dumps({"answer": str(result), "steps": [f"Evaluated expression: {expression}"]})
    except Exception as e:
        return json.dumps({"answer": f"Error calculating: {str(e)}", "steps": [f"Attempted to evaluate: {expression}"]})

tools = [
    Tool(
        name="Calculator",
        func=calculator,
        description="Evaluates simple arithmetic expressions like '2 + 3' or '5 * 4'"
    )
]

planner = load_chat_planner(llm)
executor = load_agent_executor(llm, tools, verbose=True)

agent = PlanAndExecute(planner=planner, executor=executor)

def process_query(user_input):
    try:
        if not user_input.strip():
            return "Please enter a valid query."
        response = agent.run(user_input)
        try:
            result = json.loads(response)
            agent_response = AgentResponse(**result)
            output = f"**Answer**: {agent_response.answer}\n\n**Steps**:\n" + "\n".join([f"- {step}" for step in agent_response.steps])
            return output
        except json.JSONDecodeError:
            return f"Error parsing agent response: {response}"
    except Exception as e:
        return f"Error processing request: {str(e)}"

with gr.Blocks(title="Plan and Execute Agent") as demo:
    gr.Markdown("# Simple Arithmetic Agent")
    gr.Markdown("Enter an arithmetic query, and the agent will plan and calculate the result.")
    
    user_input = gr.Textbox(
        label="Enter your query",
        placeholder="e.g., What is 2 + 3 * 4?"
    )
    
    submit_button = gr.Button("Submit")
    output = gr.Textbox(label="Agent Response", lines=10)
    
    submit_button.click(
        fn=process_query,
        inputs=user_input,
        outputs=output
    )

if __name__ == "__main__":
    demo.launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


In [6]:
import gradio as gr
from langchain_groq import ChatGroq
from langchain.agents import initialize_agent, AgentType
from langchain_experimental.tools import PythonREPLTool
from pydantic import BaseModel, Field
from typing import List
import os
import json

os.environ["GROQ_API_KEY"] = "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"

class REPLResponse(BaseModel):
    answer: str = Field(..., description="Result of the Python execution")
    steps: List[str] = Field(default_factory=list, description="Steps executed by the REPL")

llm = ChatGroq(
    api_key=os.environ["GROQ_API_KEY"],
    model_name="llama3-8b-8192",
    temperature=0.5
)

tools = [
    PythonREPLTool(
        description="Executes Python code for simple calculations or operations. Input should be a valid Python expression like '2 + 3' or 'len([1, 2, 3])'."
    )
]

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

def process_query(user_input):
    try:
        if not user_input.strip():
            return "Please enter a valid query."
        response = agent.run(
            f"Execute the following Python expression and return JSON with 'answer' and 'steps' fields: {user_input}"
        )
        try:
            result = json.loads(response)
            repl_response = REPLResponse(**result)
            output = f"*Answer: {repl_response.answer}\n\nSteps*:\n" + "\n".join([f"- {step}" for step in repl_response.steps])
            return output
        except json.JSONDecodeError:
            return f"Error parsing response: {response}"
    except Exception as e:
        return f"Error processing query: {str(e)}"

with gr.Blocks(title="Python REPL Agent") as demo:
    gr.Markdown("# Simple Python Calculator Agent")
    gr.Markdown("Enter a Python expression like '2 + 3' or 'len([1, 2, 3])' to execute.")
    user_input = gr.Textbox(
        label="Your Query",
        placeholder="e.g., 2 + 3 or len([1, 2, 3])"
    )
    submit_button = gr.Button("Execute")
    output = gr.Textbox(label="Result", lines=5)
    submit_button.click(
        fn=process_query,
        inputs=user_input,
        outputs=output
    )

if __name__ == "__main__":
    demo.launch()
    

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.
